In [69]:
import os
import pandas as pd

####################################################################################
test_set_path="/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_King"
ground_truth_csv_path="/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_King/AAPB_Test_Haystack_King_ground_truth.pt_1.csv"
####################################################################################

ground_truth = pd.read_csv(ground_truth_csv_path)
ground_truth.head(3)

,Basename,Time_in,Duration,Value,Additional_label
0,cpb-aacip-28-319s17sx3t__PRA_AAPP_BB1460_Marti...,1401,1,background,NaN
1,cpb-aacip-15-97940t7x__111486_,1045,1,"King, Martin Luther",NaN
2,cpb-aacip-15-9gm81p0j__256757_,3299,1,"King, Martin Luther",NaN


In [70]:
filename_delimiter="_Baldwin_GMM_UBM_smooth10_"
filename_delimiter='_svm_'
filename_delimiter="_GMM_UBM_smooth20_"
filename_delimiter="_svm_King_UBM_2969x2s"

svm_output_files = [os.path.join(test_set_path,item) for item in os.listdir(test_set_path) if \
                                                                            (filename_delimiter in item)&('.csv' in item)&(item[0]!='.')]

svm_output = pd.DataFrame()
list_ = []

for pathname in svm_output_files:
    try:
        basename = pathname.split('/')[-1].split(filename_delimiter)[0]
        if basename[-6:]=='.16000':
            basename = basename[:-6]
        df = pd.read_csv(pathname,index_col=None)
        df.columns=['Time_in','Numeric_ID','Duration']
        df['Basename'] = basename
        list_.append(df)
    except:
        pass      ## Because some files may be empty, we're letting errors pass.
        
        
svm_output = pd.concat(list_, ignore_index=True)
svm_output.head(3)

,Time_in,Numeric_ID,Duration,Basename
0,27,1,7,cpb-aacip-41-15bcch5q.h264
1,37,1,2,cpb-aacip-41-15bcch5q.h264
2,41,1,2,cpb-aacip-41-15bcch5q.h264


In [71]:
svm_output['Basename'][:3]

0    cpb-aacip-41-15bcch5q.h264
1    cpb-aacip-41-15bcch5q.h264
2    cpb-aacip-41-15bcch5q.h264
Name: Basename, dtype: object

In [72]:
### Making sure we're dealing with the same sets of files ###

sorted(list(set(list(ground_truth['Basename'])))) == sorted(list(set(list(svm_output['Basename']))))

True

In [73]:
### Checks classifier output table to see whether the machine thinks a given 1-second clip is the speaker ###

def classified_as_speaker(basename,time_in):
    for index, row in svm_output[svm_output['Basename']==basename].iterrows():
        row = svm_output.iloc[index]
        if float(row['Time_in']) <= float(time_in) < (float(row['Time_in'])+float(row['Duration'])):
            return True
    return False

In [74]:
### Generating confusion matrix ###

true_pos=0.0
true_neg=0.0
false_pos=0.0
false_neg=0.0


for index, row in ground_truth.iterrows():
    row = ground_truth.iloc[index]
    classifier_pos = classified_as_speaker(row['Basename'],row['Time_in'])
    ground_truth_pos = (row['Value']!='background')
    if classifier_pos == ground_truth_pos == True:
        true_pos+=1
    elif classifier_pos == ground_truth_pos == False:
        true_neg+=1
    elif (classifier_pos != ground_truth_pos) & (classifier_pos == True):
        false_pos+=1
    elif (classifier_pos != ground_truth_pos) & (classifier_pos == False):
        false_neg+=1

   

In [75]:
print(filename_delimiter)
print('\n')


print("CONFUSION MATRIX")

print(str(true_pos) +' | ' + str(false_neg))
print(str(false_pos) +' | ' + str(true_neg))

print('\nAccuracy: '+str((true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg)))
print('\n')





TP=true_pos
TN=true_neg
FP=false_pos
FN=false_neg


print('Sensitivity, hit rate, recall, or true positive rate')
TPR = TP/(TP+FN)
print(TPR)
print('\n')

print('Specificity or true negative rate')
TNR = TN/(TN+FP) 
print(TNR)
print('\n')


print('Precision or positive predictive value')
PPV = TP/(TP+FP)
print(PPV)
print('\n')


print('Negative predictive value')
NPV = TN/(TN+FN)
print(NPV)
print('\n')


print('Fall out or false positive rate')
FPR = FP/(FP+TN)
print(FPR)
print('\n')

print('False negative rate')
FNR = FN/(TP+FN)
print(FNR)
print('\n')

print('False discovery rate')
FDR = FP/(TP+FP)
print(FDR)
print('\n')

print('Overall accuracy')
ACC = (TP+TN)/(TP+FP+FN+TN)
print(ACC)
print('\n')

print('F1')
F1 = 2*((PPV*TPR)/(PPV+TPR))
print(F1)

_svm_King_UBM_2969x2s


CONFUSION MATRIX
71.0 | 56.0
16.0 | 107.0

Accuracy: 0.712


Sensitivity, hit rate, recall, or true positive rate
0.55905511811


Specificity or true negative rate
0.869918699187


Precision or positive predictive value
0.816091954023


Negative predictive value
0.656441717791


Fall out or false positive rate
0.130081300813


False negative rate
0.44094488189


False discovery rate
0.183908045977


Overall accuracy
0.712


F1
0.663551401869
